In [26]:
import sys
sys.path.append('./src')

import numpy as np
from preprocessing import load_data, preprocess_data
from sofr_swap import calculate_swap_pv, calculate_swap_sensitivity, build_portfolio_value_and_sensitivity


ImportError: cannot import name 'build_portfolio_value_and_sensitivity' from 'sofr_swap' (c:\Users\P Myat\OneDrive\Documents\GitHub\multi-asset-var-analysis\src\sofr_swap.py)

### Data Preprocessing
From **Returns of Four Stocks** & **SOFR Swap** to **Risk Factor**

In [15]:
sofr_df, AAPL, MSFT, F, BAC = load_data("data/hist_data.xlsm")
stock_returns, sofr, sofr_change, risk_factors = preprocess_data(sofr_df, AAPL, MSFT, F, BAC)

In [ ]:
matching_tenors = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']
tenors_sofr = ['1D'] + matching_tenors
latest_date = '2023-10-30'
sofr_rates = sofr.loc[latest_date].astype(float)
swap_notional = 100e6
strike = 0.042
tenors = np.arange(1, 11)

# basic swap inputs
base_rates = sofr.loc['2023-10-30'][matching_tenors].astype(float).values
base_pv = calculate_swap_pv(base_rates, np.arange(1, 11), 100e6, 0.042)

# full DV01 matrix
sofr_sensitivity, swap_price_df = calculate_swap_sensitivity(sofr, 100e6, 0.042)

# portfolio-level
P0, ak_sensitivity, relevant_factors = build_portfolio_value_and_sensitivity(sofr_sensitivity, base_pv)

Sensitivity Vector: [3.99716547e+06 7.69066531e+06 1.11655970e+07 1.44255919e+07
 1.74700654e+07 2.03058689e+07 2.29387143e+07 2.53711812e+07
 2.76083034e+07 7.35762223e+08 1.00000000e+06 1.00000000e+06
 1.00000000e+06 1.00000000e+06]


## 4. VAR model: 

## Parametric VaR Model

$$
\text{VaR} = z_{\alpha} \times \sqrt{\mathbf{w}^{T} \Sigma \mathbf{w} }
$$

Parametric VAR Model Formula:
  $$
  VaR_{\alpha,1} = |\mu_P + z_{1-\alpha} \cdot \sigma_P|
  $$


note: for the parametric VAR model's $ \mu_P $ and $ \sigma_P $, can directly use the above 3-4. Portfolio Mean & 3-5. Portfolio Variance calculation result formula to compute Parametric VAR Model's Expected P&L and Standard Deviation:
- Expected P&L ($ \mu_P $):
  $$
  \mu_P = \sum_{k=1}^{m} a^{(k)} \cdot \mu_k \cdot 1
  $$
  $ \mu_k $ = Mean of daily changes in each risk factor.
   $ h = 1 $ day 

- Portfolio Standard Deviation ($ \sigma_P $):
$$
  \sigma_P^2 = V\left[ \sum_{k=1}^{m} a^{(k)} \cdot \Delta x_{0,1d}^{(k)} \right] =  w \cdot \Sigma h \cdot w^T 
=  a \cdot \Sigma \cdot a^T 
  $$
   $ \Sigma$ = Covariance matrix of risk factor.
   
   $w$ = vector of risk factors sensitivities
   
   $a$ = vector of sensitivities $ a^{(k)} $


## Monte Carlo VaR Model - Full revaluation Approach

>##### Fit historical data to normal distribution

In [46]:
allriskfactors

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-01,-0.017698,-0.017207,0.002241,0.004430,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,-0.001949,0.032782,0.018678,0.024767,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,0.003895,0.028850,0.013966,0.005962,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.013583,0.030217,0.013083,0.003136,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877
2023-10-27,0.007938,0.005839,-0.130641,-0.037049,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,...,0.000191,0.000217,0.000240,0.000259,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384


In [31]:
mu_all = allriskfactors.mean()
sigma_all = allriskfactors.std()  
corr_mat = allriskfactors.corr().values  

>##### Ensure normality by transforming historical data

In [32]:
from statistics import NormalDist

In [33]:
n_mc = 10000
n_factors = len(allriskfactors.columns)  # Number of risk factors (stocks + SOFR)

# Step 3: Generate independent standard normal variables
uniforms = np.random.uniform(size=(n_mc, n_factors))  # Uniform variables
snorms = np.array([[NormalDist().inv_cdf(u) for u in row] for row in uniforms])
# Convert to standard normal

# Step 4: Apply Cholesky decomposition to induce correlation
factor_loadings = np.linalg.cholesky(corr_mat)  # Cholesky decomposition
snorms_correlated = np.dot(snorms, factor_loadings.T)  # Correlated risk factor changes

# Step 5: Transform standard normal samples into actual risk factor returns
simulated_returns = mu_all.values + sigma_all.values * snorms_correlated

# Step 6: Convert to DataFrame with correct column names
simulated_risk_factors = pd.DataFrame(simulated_returns, columns=allriskfactors.columns)

simulated_risk_factors.head()

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
0,-0.031228,-0.035112,-0.007035,-0.002325,-0.000535,-0.000579,-0.000321,-0.000150,-0.000623,-0.000965,...,-0.000287,-0.000246,-0.000207,-0.000173,-0.000144,-0.000121,-0.000076,-0.000045,-0.000083,-0.000050
1,-0.039122,-0.011083,0.015062,-0.003437,-0.001551,-0.000686,0.000278,0.000110,-0.000653,-0.000839,...,-0.000253,-0.000261,-0.000265,-0.000267,-0.000270,-0.000275,-0.000307,-0.000242,-0.000227,-0.000243
2,-0.010742,-0.032628,-0.001276,0.012221,0.000561,0.000134,-0.000592,-0.000314,0.000057,0.000024,...,-0.000720,-0.000743,-0.000763,-0.000781,-0.000797,-0.000811,-0.000845,-0.000835,-0.000799,-0.000751
3,0.006375,0.008400,0.024360,0.010955,-0.000679,0.000033,0.000440,0.000367,0.000372,0.000563,...,0.000688,0.000673,0.000660,0.000647,0.000628,0.000601,0.000423,0.000344,0.000182,0.000091
4,0.004573,0.000667,-0.003153,0.003621,0.000687,0.000606,0.000158,-0.000124,-0.000359,-0.000391,...,-0.000505,-0.000500,-0.000493,-0.000482,-0.000470,-0.000454,-0.000395,-0.000409,-0.000212,-0.000055


>##### Compute 1-Day Portfolio PnL for Each Simulation

$$
\text{PnL}_{\text{total}} = \text{PnL}_{\text{stocks}} + \text{PnL}_{\text{swap}}
$$


$$
\text{PnL}_{\text{swap}} = \text{PV}_{t} - \text{PV}_{t-1}
$$


#### Standard Monte Carlo Simulation 

In [34]:
# stock pnl
def pnl1d_full(simulated_risk):
    # Define the columns that correspond to stock daily returns
    stock_cols = ["AAPL daily return", 
                  "MSFT daily return", 
                  "F daily return", 
                  "BAC daily return"]
    stock_pnl = (simulated_risk[stock_cols].values * 1_000_000).sum(axis=1)
    # Calculate stock PnL: Scale daily returns by a $1M notional per stock and sum across all stocks

    #swap pnl
    swap_pnl = np.array([
        calculate_swap_pv(
            base_rates + simulated_risk[matching_tenors].iloc[i].values,# Apply interest rate shocks
            tenors, # Swap tenors from 1Y to 10Y
            swap_notional, # Notional amount of the swap
            strike # Swap fixed rate
        ) - base_pv  # PnL = PV_new - PV_old(change in present value)
        for i in range(n_mc) # Loop through each Monte Carlo simulation
    ])

    total_pnl = stock_pnl + swap_pnl
    # Total PnL is the sum of stock and swap PnL
    return stock_pnl, swap_pnl, total_pnl

# Set up initial swap valuation parameters
# Extract base rates from SOFR yield curve on a specific date (October 30, 2023)
base_rates = sofr.loc["2023-10-30"][matching_tenors].values.astype(float) 

# Define the swap tenors (1Y, 2Y, ..., 10Y)
tenors = np.arange(1, 11)         # 1Y, 2Y, ..., 10Y

# Define swap notional: $100 million
swap_notional = 100_000_000       # $100 million

# Define swap notional: $100 million
strike = 0.042                    # 4.2%

# Compute the initial swap present value (PV) before applying shocks
base_pv = calculate_swap_pv(base_rates, tenors, swap_notional, strike)

stock_pnl, swap_pnl, simulated_pnl = pnl1d_full(simulated_risk_factors)
# =================================================================

# Compute 1-day 95% Value at Risk (VaR) using Monte Carlo simulations
var1d_full_mc_95 = np.abs(np.percentile(simulated_pnl, 5))  # 5th percentile of the PnL distribution
print(f"VaR [1d, 95%], Full Revaluation: ${var1d_full_mc_95:,.2f}")

VaR [1d, 95%], Full Revaluation: $951,270.45


## Monte Carlo VaR Model - Sensitivity-Based Approach

>##### Step 1 : Define Sensitivities ($𝑎^ (𝑘)$)

In [47]:
a_k=ak_sensitivity
a_k

array([3.99716547e+06, 7.69066531e+06, 1.11655970e+07, 1.44255919e+07,
       1.74700654e+07, 2.03058689e+07, 2.29387143e+07, 2.53711812e+07,
       2.76083034e+07, 7.35762223e+08, 1.00000000e+06, 1.00000000e+06,
       1.00000000e+06, 1.00000000e+06])

>##### Step 2 : Compute Risk Factor Covariance Matrix (Σ)

Then, the resulting covariance matrix will have a shape of **(14 × 14)**:

\begin{bmatrix}
\text{cov\_stocks} & 0 \\
0 & \text{cov\_swap}
\end{bmatrix}


In [ ]:
cov_total = np.block([
    [cov_stocks, np.zeros((4, len(matching_tenors)))], 
    #top-left: stock covariance, top-right:zero matrix
    [np.zeros((len(matching_tenors), 4)), cov_swap] 
    #bottom-left:zero matrix, bottom_right:SOFR covariance
])  

>##### Step 3: Monte Carlo Simulation of Risk Factor Changes (Δ𝑋)

In [37]:
# Step 1: Generate Monte Carlo Simulations for Risk Factor Changes
n_mc = 100000  # Number of simulations
factor_loadings = np.linalg.cholesky(cov_total)  # Cholesky decomposition
simulated_risk_changes = np.random.randn(n_mc, len(a_k)) @ factor_loadings.T  # Correlated risk factor changes

# Step 2: Compute Sensitivity-Based PnL for Each Simulation
def pnl1d_sen(simulated_risk):
    return np.dot(a_k, simulated_risk)  # L = sum(a_k * ΔX)

# Generate PnL samples
pnl1d_sen_sample = [pnl1d_sen(s) for s in simulated_risk_changes]

# Step 3: Compute 1-day 95% Monte Carlo VaR (Extracting 5th percentile)
var1d_sen_mc = np.abs(np.percentile(pnl1d_sen_sample, 5))

# Print Results
print(f"Monte Carlo Sensitivity-Based Portfolio 95% VaR: ${var1d_sen_mc:,.2f}")


Monte Carlo Sensitivity-Based Portfolio 95% VaR: $1,388,501.49
